In [ ]:
import torch
import torch.nn as nn
import copy


In [ ]:


class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Linear(50, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 320)  # Flatten the output for the classifier
        x = self.classifier(x)
        return x

class DiffLrpWrapper(nn.Module):
    def __init__(self, net):
        super().__init__()
        assert isinstance(net, nn.Module), f"Expected net to be an instance of nn.Module, got {type(net)}"
        self.net = net
        self.activations = {}
        self.outputs = {}
        self._register_hooks()

    def _register_hooks(self):
        # Register a forward hook on each module
        for name, module in self.net.named_modules():
            # Avoid registering hooks on containers
            if len(list(module.children())) == 0:
                module.register_forward_hook(self._save_activation(name))

    def _save_activation(self, name):
        # This method returns a hook function
        def hook(module, input, output):
            self.activations[name] = input[0].detach()
            self.outputs[name] = output.detach()
        return hook

    def forward(self, x, target_class:torch.Tensor):
        assert x.shape[0] == target_class.shape[0], f"Expected x and target_class to have the same batch size, got {x.shape[0]} and {target_class.shape[0]}"
        # Forward pass through the network
        initial_out = self.net(x)
        # Create a mask that zeros out all elements except for the target class
        mask = torch.zeros_like(initial_out)
        mask[torch.arange(mask.size(0)), target_class] = 1  # Ensure target_class is either a scalar or has the same batch size as x

        # Apply the mask to propogate relenace forwards
        relevance = initial_out * mask
        # loop backwards from output to input layer
        for name, actual_module in list(self.net.named_modules())[::-1]:
            if len(list(actual_module.children())) == 0:
                # if the module is a leaf module, apply LRP
                # print(f"reversing layer {name} which is of type {type(actual_module)}")
                relevance = self._apply_lrp(name, actual_module, relevance.detach())
        return relevance
    

    def _apply_lrp(self, name:str, actual_module:torch.nn.Module, relevance_to_be_propagaed:torch.Tensor):
        # Get the activation of the module
        layer_activation_values = self.activations[name]
        # check datatypes coming through
        assert isinstance(layer_activation_values, torch.Tensor)
        assert isinstance(actual_module, nn.Module)
        assert isinstance(relevance_to_be_propagaed, torch.Tensor)
        # Check that the shape of the layer outputs and relevance are the same
        if not self.outputs[name].shape == relevance_to_be_propagaed.shape:
            print(f"shapes didn't match for layer {name}")
            relevance_to_be_propagaed = relevance_to_be_propagaed.view(self.outputs[name].shape)
        # Get the relevance of the output & apply LRP
        relevance = self._reverse_layer_(layer_activation_values, actual_module, relevance_to_be_propagaed)
        return relevance
    
    def _reverse_layer_(self, activations_at_start:torch.Tensor, actual_module:torch.nn.Module, relevance:torch.Tensor, epsilon=1e-9):
        # make sure corret data is coming in
        assert isinstance(activations_at_start, torch.Tensor), f"Expected activations_at_start to be a torch.Tensor, got {type(activations_at_start)}"
        assert isinstance(actual_module, nn.Module), f"Expected actual_module to be an nn.Module, got {type(actual_module)}"
        assert isinstance(relevance, torch.Tensor), f"Expected relevance to be a torch.Tensor, got {type(relevance)}"
        # print(f"activations_at_start shape: {activations_at_start.shape}")
        activations_at_start.requires_grad_()
        activations_at_start.retain_grad()
        # perform a modified forward pass (alpha beta rule applied here apparently)
        z = epsilon + actual_module.forward(activations_at_start)
        # divide the outputs by the relevance
        s = torch.div(relevance, z)
        # multiply with weights matrix and perform a backwards pass to get the unit relevance
        torch.multiply(z, s.data).sum().backward()
        # multiple activations with gradients to get the final relevance
        c = activations_at_start * activations_at_start.grad
        return c

        
    def get_activations(self):
        return self.activations



In [ ]:

# Example Usage
# Instantiate and use the wrapper
model = SimpleNet()
wrapped_model = DiffLrpWrapper(model)
target_class = torch.randint(0, 10, (20,1))

target_class

In [ ]:

# Forward pass
input_tensor = torch.randn(20, 1, 28, 28)
output = wrapped_model(input_tensor, target_class)
output.sum().backward()


In [ ]:

def print_layers(model, prefix=""):
    for name, module in model.named_children():
        if isinstance(module, nn.Sequential):
            # Recursive call to handle nested structures
            print(f"{prefix}{name} (Sequential):")
            print_layers(module, prefix=prefix + "  ")
        else:
            # Print layer type
            print(f"{prefix}{name}: {type(module)}")


In [ ]:
for name, module in wrapped_model.net.named_modules():
    print(name, type(module))

In [ ]:
if type(wrapped_model.net.features[0]) is nn.Conv2d:
    print("yes")

In [ ]:
import torch
from captum.attr import LRP

atr = LRP(wrapped_model.net)

output = wrapped_model(input_tensor)
attr = atr.attribute(input_tensor, target=0)
loss = attr.sum() - 10
loss.backward()

In [ ]:
# if one can manually define the gradient computation at run time with this function

# Step 1: Define the tensor
x = torch.tensor([2.0, 3.0], requires_grad=True)

# Step 2: Define the function, e.g., f = x^2
y = x ** 2

# Step 3: Calculate the gradient manually
grads = torch.autograd.grad(outputs=y, inputs=x, grad_outputs=torch.tensor([1.0, 1.0]))


In [ ]:
import torch
import copy

def copy_layer(layer):
    # Create a deep copy of the layer
    layer_copy = copy.deepcopy(layer)

    return layer_copy

In [ ]:
x_prime = copy_layer(wrapped_model.net.features[0])

In [ ]:
def reverse_layer(activations_at_start, layer, relevance, epsilon=1e-9):
    activations_at_start.requires_grad_()
    activations_at_start.retain_grad()
    # perform a modified forward pass (alpha beta rule applied here apparently)
    z = epsilon + copy_layer(layer).forward(activations_at_start)
    # divide the outputs by the relevance
    s = torch.div(relevance, z)
    # 
    torch.multiply(z, s.data).sum().backward()
    c = activations * activations.grad
    return c




    
    

In [ ]:
for name, module in list(wrapped_model.net.named_modules())[::-1]:
            if len(list(module.children())) == 0:
                print(f"name: {name} module: {type(module)}")
            

In [ ]:
layer = wrapped_model.net.classifier[2]
print(type(layer))
input_tensor = torch.randn(1, 1, 28, 28, requires_grad=True)
starting_relevance = wrapped_model.net(input_tensor)
activations_at_start = wrapped_model.get_activations()["classifier.1"].requires_grad_()
activations_at_start.retain_grad()
epsilon = 1e-9
# reverse_layer(activations, layer, starting_relevance)
z = epsilon + layer.forward(activations_at_start)
# divide the outputs by the relevance
s = torch.div(starting_relevance, z)
# 
torch.multiply(z, s.detach()).sum().backward()
c = activations_at_start * activations_at_start.grad

In [ ]:
type(c)